In [5]:
import modelx as mx

model = mx.read_model("Basic_Term_Model_v1")
        

ModuleNotFoundError: No module named 'modelx'

In [3]:
import json
from pathlib import Path
import datetime

def load_settings():
    """Load saved settings from a JSON file"""
    settings_file = Path("saved_settings.json")
    if not settings_file.exists():
        return None
        
    with open(settings_file, "r") as f:
        settings = json.load(f)
        if isinstance(settings.get("valuation_date"), str):
            settings["valuation_date"] = datetime.datetime.strptime(
                settings["valuation_date"], "%Y-%m-%d"
            ).date()
        return settings

load_settings()

{'valuation_date': datetime.date(2025, 1, 15),
 'assumption_table_url': 's3://valuation-model/term/run1/Assumptions.xlsx',
 'projection_period': 20,
 'output_s3_url': 's3://valuation-model/term/run1/output'}